# Dive Deeper: Supermarket Sales Analysis

> The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. [Source: Kaggle](https://www.kaggle.com/aungpyaeap/supermarket-sales)

## Import Library and Dataset

In [ ]:
# import library


Silakan import data CSV bernama `supermarket_sales.csv`. Data ini diperoleh dari Kaggle dan hanya diambil beberapa kolom untuk kebutuhan analisis.

In [ ]:
# import dataset, simpan ke object supermarket


In [ ]:
# cek informasi data


Deskripsi data:

- `invoice_id`: Computer generated sales slip invoice identification number
- `city`: Location of supercenters
- `customer`: Type of customers, recorded by Members for customers using member card and Normal for without member card
- `gender`: Gender type of customer
- `product_line`: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
- `unit_price`: Price of each product in dollar
- `quantity`: Number of products purchased by customer
- `date`: Date of purchase (Record available from January 2019 to March 2019)
- `time`: Purchase time (10am to 9pm)
- `payment`: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
- `rating`: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

## Data Pre-processing

❓ Apakah seluruh kolom di atas sudah memiliki tipe data yang sesuai? Jika belum, kolom apa saja yang perlu diubah?


In [ ]:
# cek tipe data


In [ ]:
# ubah tipe data


## Analysis

❓ Bantulah tim marketing untuk mengetahui jenis produk apa yang paling favorit di setiap kotanya? Favorit di sini berdasarkan banyaknya transaksi yang terjadi.

❓ Bantulah tim sales untuk mengetahui **total pendapatan kotor** untuk masing-masing kota, jika pada setiap transaksi diberlakukan pajak sebesar 5%.

Hint: Hitung `total` per transaksi terlebih dahulu, kemudian tambahkan dengan besar pajaknya.

❓ Bantulah tim customer relation untuk mengetahui rata-rata tingkat kepuasan customer secara bulanan untuk masing-masing kota. Apakah mengalami penurunan atau kenaikan?

___

## Self-Exploration Time ~

Pada bagian ini, kami sediakan tempat untuk Anda melakukan eksplorasi mandiri terhadap data `supermarket`. Rumuskanlah minimal **dua pertanyaan bisnis** yang menarik dari data, kemudian cobalah jawab pertanyaan tersebut menggunakan teknik-teknik yang sudah kita pelajari bersama di kelas. Sertakan insight menarik dalam bentuk narasi dan ceritakanlah di depan kelas :)

Sebagai panduan, berikut adalah teknik-teknik yang dapat Anda gunakan:

- Conditional subsetting (filter): `dataframe[kondisi]`
- Extract and transform `datetime64` component: `.dt.COMPONENT` and `.dt.to_period()`
- Frequency and aggregation table:
    - `.value_counts()`
    - `pd.crosstab()`
    - `pd.pivot_table()`
- Sorting table: `.sort_values()`

**Pertanyaan 1:** ...

In [ ]:
# code here


> **📈 Insight:** ...

**Pertanyaan 2:** ...

In [ ]:
# code here


> **📈 Insight:** ...